In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv("C:\\Users\\KHAN\\Downloads\\supply_chain_data.csv")


In [3]:
df.head()


,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [4]:
df.info()

# Check for missing values
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             100 non-null    object 
 1   SKU                      100 non-null    object 
 2   Price                    100 non-null    float64
 3   Availability             100 non-null    int64  
 4   Number of products sold  100 non-null    int64  
 5   Revenue generated        100 non-null    float64
 6   Customer demographics    100 non-null    object 
 7   Stock levels             100 non-null    int64  
 8   Lead times               100 non-null    int64  
 9   Order quantities         100 non-null    int64  
 10  Shipping times           100 non-null    int64  
 11  Shipping carriers        100 non-null    object 
 12  Shipping costs           100 non-null    float64
 13  Supplier name            100 non-null    object 
 14  Location                 10

Product type               0
SKU                        0
Price                      0
Availability               0
Number of products sold    0
Revenue generated          0
Customer demographics      0
Stock levels               0
Lead times                 0
Order quantities           0
Shipping times             0
Shipping carriers          0
Shipping costs             0
Supplier name              0
Location                   0
Lead time                  0
Production volumes         0
Manufacturing lead time    0
Manufacturing costs        0
Inspection results         0
Defect rates               0
Transportation modes       0
Routes                     0
Costs                      0
dtype: int64

In [5]:
# Convert column names to snake_case
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")


# Create Structured Datasets

 Inventory Data

In [6]:
inventory_df = df[[
    'sku', 'product_type', 'price', 'availability',
    'stock_levels', 'lead_times', 'revenue_generated',
    'number_of_products_sold'
]]


Order Fulfillment Data

In [7]:
order_df = df[[
    'sku', 'order_quantities', 'number_of_products_sold',
    'shipping_times', 'availability'
]].copy()

# Add a column for on-time delivery (assuming ≤ 3 days is on-time)
order_df['on_time_delivery'] = order_df['shipping_times'].apply(lambda x: 'Yes' if x <= 3 else 'No')


Supplier Performance Data

In [8]:
supplier_df = df[[
    'sku', 'supplier_name', 'location', 'lead_time',
    'defect_rates', 'production_volumes',
    'manufacturing_lead_time', 'manufacturing_costs',
    'inspection_results'
]]


Transportation Data

In [9]:
transport_df = df[[
    'sku', 'shipping_carriers', 'shipping_times',
    'transportation_modes', 'routes', 'shipping_costs'
]]


Cost Data

In [10]:
cost_df = df[[
    'sku', 'costs', 'manufacturing_costs', 'shipping_costs'
]]


Export the Cleaned Files

In [11]:
inventory_df.to_csv("inventory_data.csv", index=False)
order_df.to_csv("order_data.csv", index=False)
supplier_df.to_csv("supplier_data.csv", index=False)
transport_df.to_csv("transport_data.csv", index=False)
cost_df.to_csv("cost_data.csv", index=False)


In [13]:
import pandas as pd
import numpy as np


In [14]:

inventory_df = pd.read_csv("C:\\Users\\KHAN\\Desktop\\supply\\inventory_data.csv")
order_df = pd.read_csv("C:\\Users\\KHAN\\Desktop\\supply\\order_data.csv")
supplier_df = pd.read_csv("C:\\Users\\KHAN\\Desktop\\supply\\supplier_data.csv")
transport_df = pd.read_csv("C:\\Users\\KHAN\\Desktop\\supply\\transport_data.csv")
cost_df = pd.read_csv("C:\\Users\\KHAN\\Desktop\\supply\\cost_data.csv")


In [15]:
# Inventory Data: Add fake date + warehouse from supplier
inventory_df['date'] = pd.date_range(start='2024-01-01', periods=len(inventory_df), freq='D')

# Load supplier data again to merge location as 'warehouse'
supplier_info = supplier_df[['sku', 'location']].copy()
inventory_df = pd.merge(inventory_df, supplier_info, on='sku', how='left')
inventory_df.rename(columns={'location': 'warehouse'}, inplace=True)

# Order Data: Add date + fake customer_id
order_df['date'] = pd.date_range(start='2024-01-01', periods=len(order_df), freq='D')
order_df['customer_id'] = ['CUST' + str(i).zfill(3) for i in range(len(order_df))]

# Supplier Data: Add fake date
supplier_df['date'] = pd.date_range(start='2024-01-01', periods=len(supplier_df), freq='D')

# Transport Data: Add date + delivery_status
transport_df['date'] = pd.date_range(start='2024-01-01', periods=len(transport_df), freq='D')
transport_df['delivery_status'] = transport_df['shipping_times'].apply(lambda x: 'On Time' if x <= 3 else 'Delayed')

# Cost Data: Convert to long format + add date
cost_long_df = pd.melt(cost_df, id_vars=['sku'], 
                       value_vars=['costs', 'shipping_costs', 'manufacturing_costs'],
                       var_name='cost_type', value_name='amount')
cost_long_df['date'] = pd.date_range(start='2024-01-01', periods=len(cost_long_df), freq='D')


In [16]:
inventory_df.to_csv("inventory_data.csv", index=False)
order_df.to_csv("order_data.csv", index=False)
supplier_df.to_csv("supplier_data.csv", index=False)
transport_df.to_csv("transport_data.csv", index=False)
cost_long_df.to_csv("cost_data.csv", index=False)
